<a href="https://colab.research.google.com/github/pedropberger/code-misc/blob/main/NLP_MPES_PI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NLP MPES PI

Roadmap

- Import data
- ETL
- Html clean
- Token
- Embedding
- Relation matrix

# Packages

In [1]:

# Data
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

# Tools
from google.colab import drive
import os
from bs4 import BeautifulSoup

# NLP

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import re


# Upload and Clean Data

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Assuming your data file is a CSV file, change the format accordingly if it's different
file_path = '/content/drive/My Drive/Colab Notebooks/Data/NLP_PI_Beta'

# Load the data into a DataFrame

db_full =  pd.read_parquet(file_path)

# Now you can work with the data
print(db_full.head())  # Display the first few rows of the data


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
    IdAuto          Numero  IdTipoAuto  id_tipo_documento  \
0  3165243  20230000118398         2.0                 17   
1  3165533  20230000147307         2.0                 17   
2  3165920  20230000186102        15.0                 17   
3  3166481  20230000242268         5.0                 17   
4  3164456  20230000039664         2.0                 17   

              TituloDoc                                      conteudo_html  
0                  None  <p style="text-align: center;"><img src="data:...  
1                  None  <p style="text-align: center;"><img src="data:...  
2                  None  <p style="text-align: center;"><img src="data:...  
3  - Ação Civil Pública  <p style="text-align: center;"><img src="data:...  
4       Petição Inicial  <p style="text-align: center;"><img src="data:...  


# HTML Cleaning

In [3]:
# Fase 2 - Cleaning HTML

def extract_text_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')

    # Add space between words (after we will remove then in the tokenization fase)
    text_with_spaces = ' '.join(soup.stripped_strings)

    return text_with_spaces

# Aplicando a função à coluna 'conteudo_html'
db_full['conteudo_text'] = db_full['conteudo_html'].apply(extract_text_from_html)

# Exibindo o DataFrame resultante
print(db_full.head())

print('Dropando a coluna "conteudo_html"')
db_full.drop('conteudo_html', axis=1, inplace=True)

db_full.head()

    IdAuto          Numero  IdTipoAuto  id_tipo_documento  \
0  3165243  20230000118398         2.0                 17   
1  3165533  20230000147307         2.0                 17   
2  3165920  20230000186102        15.0                 17   
3  3166481  20230000242268         5.0                 17   
4  3164456  20230000039664         2.0                 17   

              TituloDoc                                      conteudo_html  \
0                  None  <p style="text-align: center;"><img src="data:...   
1                  None  <p style="text-align: center;"><img src="data:...   
2                  None  <p style="text-align: center;"><img src="data:...   
3  - Ação Civil Pública  <p style="text-align: center;"><img src="data:...   
4       Petição Inicial  <p style="text-align: center;"><img src="data:...   

                                       conteudo_text  
0  Ministério Público do Estado do Espírito Santo...  
1  Ministério Público do Estado do Espírito Santo...  

,IdAuto,Numero,IdTipoAuto,id_tipo_documento,TituloDoc,conteudo_text
0,3165243,20230000118398,2.0,17,None,Ministério Público do Estado do Espírito Santo...
1,3165533,20230000147307,2.0,17,None,Ministério Público do Estado do Espírito Santo...
2,3165920,20230000186102,15.0,17,None,Ministério Público do Estado do Espírito Santo...
3,3166481,20230000242268,5.0,17,- Ação Civil Pública,Ministério Público do Estado do Espírito Santo...
4,3164456,20230000039664,2.0,17,Petição Inicial,Ministério Público do Estado do Espírito Santo...


In [4]:
print(db_full.loc[1, 'conteudo_text'])

Ministério Público do Estado do Espírito Santo Promotoria de Justiça de Conceição do Castelo 1º Promotor de Justiça GAMPES: 2023.0000.1473-07 EXMO. SR. DR. JUIZ DE DIREITO DA COMARCA INTEGRADA CONCEIÇÃO DO CASTELO/BREJETUBA - ESTADO DO ESPÍRITO SANTO. O MINISTÉRIO PÚBLICO DO ESTADO DO ESPÍRITO SANTO, por meio de seu órgão de execução abaixo assinado, legitimado pelos artigos 1º, III, 5º “caput”, 6º, 127, 129, inciso III, 196 e 197 todos da Constituição da República Federativa do Brasil de 1988 e com fundamento nas Leis Federais nº 6.938/81, 7.347/85 e Lei 8080/90 vem perante Vossa Excelência propor a presente AÇÃO CIVIL PÚBLICA COM PEDIDO DE TUTELA DE URGÊNCIA em face do: MUNICÍPIO DE BREJETUBA/ES , C NPJ nº 01.612.674/0001-00, pessoa jurídica de direito público interno, com sede na Avenida Ângelo Uliana, s/nº, bairro Uliana, Brejetuba/ES, CEP: 29630-000, telefone (27) 3733-1200, e -mail: gabinete@brejetuba.es.gov.br, representado pelo Exmo. Sr Prefeito Levi Marques de Souza; e ESTADO 

# Tokenizer

In [5]:

# Downloading packs

nltk.download('stopwords')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:

# Função para tokenizar o texto removendo stopwords
def tokenize_text(text):
    tokens = word_tokenize(text.lower())  # Tokenização e conversão para minúsculas

    #tokens = [token for token in tokens if token.isalpha()]  # Remover pontuações e números
    stop_words = set(stopwords.words('portuguese')) # Remover pontuações e números
    tokens = [token for token in tokens if token not in stop_words]  # Remover stopwords

    # Identificar padrões de "Lei nº número/ano xx.xxx/xxxx|xxxxx/xx||" e combinar o número com a palavra "lei"
    for i, token in enumerate(tokens):
        if token == "lei" and i < len(tokens) - 2 and re.match(r'\d{1,5}\.\d{3}/\d{4}|\d{1,5}/\d{1,4}', tokens[i+2]):
            numero_lei = re.sub(r'\D', '', tokens[i+2])  # Extrair apenas os dígitos do número da lei
            tokens[i] = f"lei{numero_lei}"
            del tokens[i+1:i+3]  # Remover os tokens "nº" e o número/ano da lei

    # Identificar padrões de "Lei número/ano xx.xxx/xxxx|xxxxx/xx||" e combinar o número com a palavra "lei"
    for i, token in enumerate(tokens):
        if token == "lei" and i < len(tokens) - 1 and re.match(r'\d{1,5}\.\d{3}/\d{4}|\d{1,5}/\d{1,4}', tokens[i+1]):
            numero_lei = re.sub(r'\D', '', tokens[i+1])  # Extrair apenas os dígitos do número da lei
            tokens[i] = f"lei{numero_lei}"
            del tokens[i+1:i+2]  # Remover os tokens "nº" e o número/ano da lei


    # Remover pontuações e símbolos avulsos
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]

    # Remover tokens vazios
    tokens = [token for token in tokens if token.strip()]

    return tokens


string = "A Lei Nº 12.527/2011 e a lei nº 2527/11 e a lei 2527/11 e a lei n 27/2017 são maravilhosas e bacanas."
token_test = tokenize_text(string)
print(token_test)

['lei125272011', 'lei252711', 'lei252711', 'lei272017', 'maravilhosas', 'bacanas']


In [7]:
# Aplicar a função de tokenização à coluna de texto
db_full['token'] = db_full['conteudo_text'].apply(tokenize_text)

# Exibir o DataFrame resultante
db_full.head()

,IdAuto,Numero,IdTipoAuto,id_tipo_documento,TituloDoc,conteudo_text,token
0,3165243,20230000118398,2.0,17,None,Ministério Público do Estado do Espírito Santo...,"[ministério, público, estado, espírito, santo,..."
1,3165533,20230000147307,2.0,17,None,Ministério Público do Estado do Espírito Santo...,"[ministério, público, estado, espírito, santo,..."
2,3165920,20230000186102,15.0,17,None,Ministério Público do Estado do Espírito Santo...,"[ministério, público, estado, espírito, santo,..."
3,3166481,20230000242268,5.0,17,- Ação Civil Pública,Ministério Público do Estado do Espírito Santo...,"[ministério, público, estado, espírito, santo,..."
4,3164456,20230000039664,2.0,17,Petição Inicial,Ministério Público do Estado do Espírito Santo...,"[ministério, público, estado, espírito, santo,..."


In [8]:
print(db_full.loc[1, 'token'])

['ministério', 'público', 'estado', 'espírito', 'santo', 'promotoria', 'justiça', 'conceição', 'castelo', '1º', 'promotor', 'justiça', 'gampes', '20230000147307', 'exmo', 'sr', 'dr', 'juiz', 'direito', 'comarca', 'integrada', 'conceição', 'castelobrejetuba', 'estado', 'espírito', 'santo', 'ministério', 'público', 'estado', 'espírito', 'santo', 'meio', 'órgão', 'execução', 'abaixo', 'assinado', 'legitimado', 'artigos', '1º', 'iii', '5º', 'caput', '6º', '127', '129', 'inciso', 'iii', '196', '197', 'todos', 'constituição', 'república', 'federativa', 'brasil', '1988', 'fundamento', 'leis', 'federais', 'nº', '693881', '734785', 'lei808090', 'vem', 'perante', 'vossa', 'excelência', 'propor', 'presente', 'ação', 'civil', 'pública', 'pedido', 'tutela', 'urgência', 'face', 'município', 'brejetubaes', 'c', 'npj', 'nº', '01612674000100', 'pessoa', 'jurídica', 'direito', 'público', 'interno', 'sede', 'avenida', 'ângelo', 'uliana', 'snº', 'bairro', 'uliana', 'brejetubaes', 'cep', '29630000', 'telef

# Token data exploration

In [9]:

# Tokenize the text (assuming it's already tokenized, otherwise you can use NLTK or SpaCy for tokenization)
tokens = [token for sublist in db_full['token'] for token in sublist]

# Count the frequency of each token
token_freq = Counter(tokens)



In [10]:
print((token_freq))

Counter({'art': 54206, 'público': 52013, 'direito': 51422, 'saúde': 44748, 'estado': 41604, 'nº': 40585, 'ministério': 38285, 'adolescente': 35205, 'lei': 35198, 'justiça': 34043, 'criança': 32607, 'artigo': 31795, 'santo': 21656, 'espírito': 21349, 'presente': 21108, 'forma': 20273, 'representado': 19484, 'medida': 19361, 'caso': 19328, 'ação': 19223, 'proteção': 18518, 'direitos': 18085, 'conforme': 17526, 'ii': 17356, 'termos': 16618, 'i': 16519, 'tratamento': 16056, 'pública': 15855, 'tutela': 15642, 'constituição': 15578, 'civil': 15429, 'medidas': 15142, 'poder': 14504, 'federal': 14493, 'promotor': 14062, 'pessoa': 14059, 'sendo': 13848, 'estatuto': 13540, 'internação': 13517, 'autos': 13467, '1º': 13292, 'fatos': 13243, 'bem': 13227, 'processo': 13209, 'assim': 12966, 'iii': 12818, 'vida': 12726, 'situação': 12631, 'familiar': 12452, 'penal': 12298, 'todos': 12143, 'ainda': 12102, 'família': 12046, 'social': 11870, 'valor': 11859, 'execução': 11750, 'risco': 11548, 'multa': 109

In [11]:
# prompt: export token_freq as xlsx with colums token and frequency

xls_path = '/content/drive/My Drive/Colab Notebooks/Data/token_freq.xlsx'

# Create a DataFrame from the token frequencies
token_freq_df = pd.DataFrame(token_freq.items(), columns=['token', 'frequency'])

# Save the DataFrame to an Excel file
token_freq_df.to_excel(xls_path, index=False)


In [12]:

# Visualize the token frequencies (optional)
most_common_tokens = token_freq.most_common(20)  # Select the 20 most common tokens
tokens, frequencies = zip(*most_common_tokens)  # Unpack tokens and frequencies

print(most_common_tokens)

[('art', 54206), ('público', 52013), ('direito', 51422), ('saúde', 44748), ('estado', 41604), ('nº', 40585), ('ministério', 38285), ('adolescente', 35205), ('lei', 35198), ('justiça', 34043), ('criança', 32607), ('artigo', 31795), ('santo', 21656), ('espírito', 21349), ('presente', 21108), ('forma', 20273), ('representado', 19484), ('medida', 19361), ('caso', 19328), ('ação', 19223)]


# Token Cleasing

In [13]:
# Create a df called db_token from db_full with only the collums idauto and token

db_token = db_full[['IdAuto', 'token']]


In [14]:
db_token.head()

,IdAuto,token
0,3165243,"[ministério, público, estado, espírito, santo,..."
1,3165533,"[ministério, público, estado, espírito, santo,..."
2,3165920,"[ministério, público, estado, espírito, santo,..."
3,3166481,"[ministério, público, estado, espírito, santo,..."
4,3164456,"[ministério, público, estado, espírito, santo,..."


In [15]:
# Remove tokens with low frequency

freq_min = 3

# Get the list of tokens with frequency less than freq
low_freq_tokens = [token for token, freq in token_freq.items() if freq < freq_min]

# Convert low_freq_tokens to a set for faster membership testing
low_freq_tokens_set = set(low_freq_tokens)

# Function to remove low-frequency tokens
def remove_low_freq_tokens(tokens):
    return [token for token in tokens if token not in low_freq_tokens_set]

# Apply the function to each row in 'token' column
db_token.loc[:, 'token'] = db_token['token'].apply(remove_low_freq_tokens)

# Print the updated DataFrame
db_token.head()

,IdAuto,token
0,3165243,"[ministério, público, estado, espírito, santo,..."
1,3165533,"[ministério, público, estado, espírito, santo,..."
2,3165920,"[ministério, público, estado, espírito, santo,..."
3,3166481,"[ministério, público, estado, espírito, santo,..."
4,3164456,"[ministério, público, estado, espírito, santo,..."


In [16]:
# Remove short tokens

token_short_len = 3

# Function to remove short tokens
def remove_short_tokens(tokens):
    return [token for token in tokens if len(token) >= token_short_len]

# Apply the function to each row in 'token' column
db_token.loc[:, 'token'] = db_token['token'].apply(remove_short_tokens)

# Print the updated DataFrame
print(db_token.head())

    IdAuto                                              token
0  3165243  [ministério, público, estado, espírito, santo,...
1  3165533  [ministério, público, estado, espírito, santo,...
2  3165920  [ministério, público, estado, espírito, santo,...
3  3166481  [ministério, público, estado, espírito, santo,...
4  3164456  [ministério, público, estado, espírito, santo,...


In [17]:
# Tokenize the text (assuming it's already tokenized, otherwise you can use NLTK or SpaCy for tokenization)
tokens = [token for sublist in db_token['token'] for token in sublist]

# Count the frequency of each token
token_freq = Counter(tokens)

print(len(token_freq))

44030


In [18]:
len(db_token)

7488

# TF-IDF

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np